In [ ]:
import os
import pathlib

# Install dependencies
LOCAL_DATABRICKS_NOTEBOOK_PATH = os.getenv('LOCAL_DATABRICKS_NOTEBOOK_PATH')
if LOCAL_DATABRICKS_NOTEBOOK_PATH and pathlib.Path(LOCAL_DATABRICKS_NOTEBOOK_PATH).exists():
    print(f"Installing databricks-notebook from {LOCAL_DATABRICKS_NOTEBOOK_PATH}")
    %pip install --editable "{LOCAL_DATABRICKS_NOTEBOOK_PATH}"
else:
    print("Installing databricks-notebook from git")
    %pip install git+https://github.com/datafold/databricks-notebook.git

# Restart to make dependencies available
# %restart_python on databricks notebook
import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True)


In [ ]:
org_token = "token1"

from databricks_notebook import create_organization
api_token, org_id = create_organization(org_token)

In [ ]:
queries_to_translate = [
"""
-- Monthly Sales Analysis Report
-- Analyzes sales trends with category rankings

drop table if exists DF_RESULT_QUERY_1_SQL_2234AC5;
create table DF_RESULT_QUERY_1_SQL_2234AC5 AS (
    WITH monthly_sales_2234AC5 AS (
        SELECT
            PRODUCT_ID,
            PRODUCT_NAME,
            CATEGORY,
            DATE_TRUNC('MONTH', SALE_DATE) AS SALE_MONTH,
            SUM(QUANTITY) AS TOTAL_QUANTITY,
            SUM(T_REVENUE) AS T_REVENUE,
            COUNT(DISTINCT CUSTOMER_ID) AS UNIQUE_CUSTOMERS
        FROM DMA_TRANSLATIONS.DATAFOLD_TMP.SALES_TRANSACTIONS_2234AC5
        GROUP BY
            PRODUCT_ID,
            PRODUCT_NAME,
            CATEGORY,
            DATE_TRUNC('MONTH', SALE_DATE)
    )
    SELECT
        PRODUCT_ID,
        PRODUCT_NAME,
        CATEGORY,
        SALE_MONTH,
        TOTAL_QUANTITY,
        T_REVENUE,
        UNIQUE_CUSTOMERS,
        -- Rank products by t_revenue within each category and month
        RANK() OVER (
            PARTITION BY CATEGORY, SALE_MONTH
            ORDER BY T_REVENUE DESC
        ) AS CATEGORY_RANK
    FROM monthly_sales_2234AC5
    QUALIFY CATEGORY_RANK <= 3
    ORDER BY
        SALE_MONTH DESC,
        CATEGORY,
        T_REVENUE DESC
);
"""
]

from databricks_notebook import translate_queries, view_translation_results_as_html

project_id, translation_id = translate_queries(api_token, queries_to_translate)
html = view_translation_results_as_html(api_token, project_id, translation_id)

from IPython.display import HTML, display
display(HTML(html))